In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path 
import re
import string

In [2]:
df= pd.read_csv(r'/Users/lolitalolita/Downloads/recipes.csv')

In [3]:
df

,RecipeId,Name,RecipeCategory,Calories,CholesterolContent,CarbohydrateContent,SugarContent,ProteinContent,RecipeServings,HighScore
0,46085,Crock Pot Baked Potato Soup,One Dish Meal,699.8,137.3,46.1,1.4,20.9,6,1
1,93832,Frittata Di Spaghetti (spaghetti Frittata),Breakfast,297.1,191.8,11.7,0.7,12.2,8,1
2,36034,Berries With Italian Cream,Dessert,131.9,23.3,10.3,4.4,9.1,6,0
3,329988,Pork Tenderloin Medallions With Fresh Figs,< 15 Mins,203.0,74.8,1.5,0.6,23.3,4,1
4,59886,Kaseropita (Tiropita Using Kaseri Cheese),Savory Pies,261.6,103.6,20.9,0.2,6.7,15,1
...,...,...,...,...,...,...,...,...,...,...
43087,22945,Yummy Meatloaf,Meat,281.5,111.3,13.2,11.1,25.3,6,1
43088,110033,Slow Cooker Shanks,Lamb/Sheep,697.2,242.1,3.8,1.9,72.9,4,0
43089,240945,Sin-Free Chocolate Chews,Drop Cookies,94.5,0.0,22.7,19.1,2.0,14,1
43090,164157,Montreal Salsa Chicken,Chicken Breast,207.1,72.6,20.1,12.7,24.9,4,1


In [4]:
# Get first 1000 recipes 
df=df[:1000]

In [5]:
# Get calories and content per one serving
df_per_serving=df[['Calories','CholesterolContent','CarbohydrateContent','SugarContent','ProteinContent']].div(df.RecipeServings, axis=0) 

In [6]:
df_per_serving

,Calories,CholesterolContent,CarbohydrateContent,SugarContent,ProteinContent
0,116.633333,22.883333,7.683333,0.233333,3.483333
1,37.137500,23.975000,1.462500,0.087500,1.525000
2,21.983333,3.883333,1.716667,0.733333,1.516667
3,50.750000,18.700000,0.375000,0.150000,5.825000
4,17.440000,6.906667,1.393333,0.013333,0.446667
...,...,...,...,...,...
995,371.700000,393.200000,27.800000,2.500000,20.600000
996,46.366667,0.000000,7.066667,5.925000,0.233333
997,21.366667,5.125000,2.508333,1.566667,0.266667
998,119.000000,0.000000,3.200000,2.400000,0.100000


In [7]:
# Import PuLP modeler functions
from pulp import *

In [8]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Minimal_sugar_intake", LpMinimize)

/Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [9]:
Name_dict = list(df['Name'])

Calories_dict = dict(zip(Name_dict, df_per_serving['Calories']))
CholesterolContent_dict = dict(zip(Name_dict , df_per_serving['CholesterolContent']))
CarbohydrateContent_dict = dict(zip(Name_dict , df_per_serving['CarbohydrateContent']))
SugarContent_dict = dict(zip(Name_dict, df_per_serving['SugarContent']))
ProteinContent_dict = dict(zip(Name_dict, df_per_serving['ProteinContent']))

In [10]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
recipe_name_vars = LpVariable.dicts("R", Name_dict, 0)

In [11]:
# The objective function is added to 'prob' first
prob += (
    lpSum([SugarContent_dict[i] * recipe_name_vars[i] for i in Name_dict]),
    "Total sugar content per dish",)

In [12]:
# The constraints are added to 'prob'

prob += (
    lpSum([Calories_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 2000.0,
    "Calories_dict_Requirement_max",
)
prob += (
    lpSum([Calories_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 1500.0,
    "Calories_dict_Requirement_min",
)


prob += (
    lpSum([CholesterolContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 13.0,
    "CholesterolContent_Requirement_max",
)
prob += (
    lpSum([CholesterolContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 5.0,
    "CholesterolContent_Requirement_min",
)

prob += (
    lpSum([CarbohydrateContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 150.0,
    "CarbohydrateContent_Requirement_max",
)

prob += (
    lpSum([CarbohydrateContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 120.0,
    "CarbohydrateContent_Requirement_min",
)

prob += (
    lpSum([ProteinContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 112.0,
    "ProteinContent_Requirement_max",
)
prob += (
    lpSum([ProteinContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 70.0,
    "ProteinContent_Requirement_min",
)

In [13]:
# The problem data is written to an .lp file
prob.writeLP("RecipeModel.lp")

[R_"All_in_the_Kitchen"_Chili,
 R_(Fake)_Crab_Stroganoff,
 R_*s.w.o.c.*_Pie_Cookies,
 R_3_Dragon’s_Szechuan_Steak,
 R_4_Bean_Relish,
 R_5_Ingredient_Crescent_Roll_Breakfast_Bake,
 R_5_Layer_Italian_Dip,
 R_7_Minute_Rich_and_Creamy_Polenta,
 R_99_Bananas_and_a_Coconut_Cocktail,
 R_A_Leg_up_(Oven_Fried_Drumsticks),
 R_Aam_Lhassi,
 R_Aaron_Tippin's_Mexican_Casserole,
 R_Acorn_Squash_Soup,
 R_Addictive_Chocolate_Tapioca_Pudding,
 R_All_American_Pancakes,
 R_Almond_Crusted_Chicken_Breast,
 R_Almond_Shortbreads,
 R_Amazing_Apple_Pie,
 R_Amazing_Baked_Lemon_Garlic_Chicken_Thighs_and_Potatoes,
 R_American_Girl_Magazine's_Colorful_Cake,
 R_An_Oatmeal_Poached_Egg,
 R_Annabel's_Homemade_Granola,
 R_Annie's_Turkey_Salad,
 R_Antipasto_Squares,
 R_Anytime_Pizza_Casserole,
 R_Apple_Cinnamon_Oatmeal,
 R_Apple_Oven_Pancake,
 R_Apple_Romaine_Salad,
 R_Apple_Strawberry_Rhubarb_Crisp,
 R_Apple_Stuffed_Tenderloin_With_Cinnamon_Raisin_Sauce,
 R_Apple_Upside_Down_Cake,
 R_Apple_Wrap_Sandwich,
 R_Apples_and_G

In [14]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/89b9a5596a8649c88be73721462bc056-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/89b9a5596a8649c88be73721462bc056-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 8490 RHS
At line 8499 BOUNDS
At line 8500 ENDATA
Problem MODEL has 8 rows, 993 columns and 7498 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-1) rows, 989 (-4) columns and 6501 (-997) elements
0  Obj 0 Primal inf 304.32181 (3)
3  Obj 2.7512142e-16
Optimal - objective value 0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 3 iterations time 0.002, Presolve 0.00
Option for pr

1

In [15]:
# The status of thesolution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [16]:
# Each of the variables is printed with it's resolved optimum value
    
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

R_Coffee_for_the_Damned = 2.0890708
R_Danish_Coffee = 34.488914
R_Grilled_Cajun_Chicken = 0.66225166
R_Spelt_Pizza_Crust = 273.06651


In [17]:
# The optimised objective function value is printed to the screen
print("Minimal sugar intake = ", value(prob.objective))

Minimal sugar intake =  0.0
